In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import torch.nn.functional as F
import os
import imblearn
import numpy as np
from torch.utils.data import TensorDataset

In [2]:
class Model(nn.Module):
    def __init__(self, num_classes=7):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 256, kernel_size=3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(256)
        self.relu2 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.3)

        self.conv3 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(512)
        self.relu3 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout(0.3)

        self.conv4 = nn.Conv2d(512, 768, kernel_size=3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(768)
        self.relu4 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout(0.3)

        self.conv5 = nn.Conv2d(768, 1024, kernel_size=3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(1024)
        self.relu5 = nn.ReLU()
        self.maxpool4 = nn.MaxPool2d(2, 2)
        self.dropout4 = nn.Dropout(0.3)


        self.flatten = nn.Flatten()


        self.fc1 = nn.Linear(1024*3*3, 1024)
        self.batchnorm6 = nn.BatchNorm1d(1024)
        self.relu7 = nn.ReLU()
        self.dropout6 = nn.Dropout(0.25)

        self.fc2 = nn.Linear(1024, num_classes)
        # self.relu8 = nn.ReLU()
        # self.batchnorm7 = nn.BatchNorm1d(128)
        # self.dropout7 = nn.Dropout(0.3)

        # self.fc3= nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.batchnorm1(x)
        x = self.relu2(x)
        x = self.maxpool1(x)
        x = self.dropout1(x)

        x = self.conv3(x)
        x = self.batchnorm2(x)
        x = self.relu3(x)
        x = self.maxpool2(x)
        x = self.dropout2(x)

        x = self.conv4(x)
        x = self.batchnorm3(x)
        x = self.relu4(x)
        x = self.maxpool3(x)
        x = self.dropout3(x)

        x = self.conv5(x)
        x = self.batchnorm4(x)
        x = self.relu5(x)
        x = self.maxpool4(x)
        x = self.dropout4(x)

        # x = self.conv6(x)
        # x = self.batchnorm5(x)
        # x = self.relu6(x)
        # x = self.maxpool5(x)
        # x = self.dropout5(x)

        x = self.flatten(x)

        x = self.fc1(x)
        x = self.batchnorm6(x)
        x = self.relu7(x)
        x = self.dropout6(x)

        x = self.fc2(x)
        x = self.softmax(x)

        return x

In [3]:
log_dir = './log'  # 훈련 로그 디렉토리
train_dataset_path = "./input/data/train"  # 훈련 데이터셋 경로
val_dataset_path = "./input/data/val"
test_dataset_path = "./input/data/test"  # 테스트 데이터셋 경로
batch_size = 128
lr = 1e-3
epochs = 720
num_classes = 7  # 분류 클래스 수

모델 직접 생성

In [ ]:

# 모델 생성
model = Model()

# 모델을 GPU 또는 CPU로 이동
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

# 옵티마이저, 손실 함수, 스케줄러 정의
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

# TensorBoard를 사용하여 훈련 로그 기록
writer = SummaryWriter(log_dir)

완성된 모델 불러오기

In [4]:
# 모델을 GPU 또는 CPU로 이동
model = Model()
model.load_state_dict(torch.load('./model/mymodel3.h5'))

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

# 옵티마이저, 손실 함수, 스케줄러 정의
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

In [5]:
# 훈련 데이터셋 변환
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder(train_dataset_path, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# # train_dataset에서 데이터 가져오기
# all_images = []
# all_labels = []
# for images, labels in train_dataset:
#     all_images.append(images.numpy())
#     all_labels.append(labels)

# np_all_images = np.concatenate(all_images, axis=0)
# np_all_images = np_all_images.reshape(28709, 48*48)
# # 오버 샘플링 적용
# smote = imblearn.over_sampling.SMOTE()
# oversampled_images, oversampled_labels = smote.fit_resample(np_all_images, all_labels)
# oversampled_labels = np.array(oversampled_labels)
# # PyTorch 데이터셋으로 변환
# from torch.utils.data import TensorDataset

# oversampled_images = torch.from_numpy(oversampled_images.reshape(50505,1,48,48)).float()
# oversampled_labels = torch.from_numpy(oversampled_labels)

# oversampled_dataset = TensorDataset(oversampled_images, oversampled_labels)
# oversampled_loader = DataLoader(oversampled_dataset, batch_size=batch_size, shuffle=True)


# 검증 데이터셋 변환
val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

val_dataset = datasets.ImageFolder(val_dataset_path, transform=val_transforms)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# 테스트 데이터셋 변환
test_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_dataset = datasets.ImageFolder(test_dataset_path, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [6]:
def train():

    # 조기 종료를 위한 변수 정의
    best_acc = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(iter(train_loader)):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model.forward(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            #scheduler.step(loss)
            train_loss += loss.item()
        # 검증
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in tqdm(iter(val_loader)):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        # 훈련 및 검증 손실, 정확도 기록
        writer.add_scalar('Loss/train', train_loss / len(train_loader), epoch)
        writer.add_scalar('Loss/val', val_loss / len(val_loader), epoch)
        writer.add_scalar('Accuracy/val', 100 * correct / total, epoch)

        if best_acc < 100 * correct / total:
            best_acc = 100 * correct / total
            torch.save(model.state_dict(),'./besmodel.h5')


        print(f'epoch [{epoch + 1}/{epochs}], '
            f'train_loss: {train_loss / len(train_loader):.4f}, '
            f'val_loss: {val_loss / len(val_loader):.4f}, '
              f'accuracy: {100 * correct / total:.2f}%')



In [7]:
# 테스트
def test():
  model.eval()
  test_loss = 0.0
  correct = 0
  total = 0
  with torch.no_grad():
      for images, labels in tqdm(iter(test_loader)):
          images, labels = images.to(device), labels.to(device)
          outputs = model.forward(images)
          loss = criterion(outputs, labels)
          test_loss += loss.item()
          _, predicted = outputs.max(1)
          total += labels.size(0)
          correct += predicted.eq(labels).sum().item()

  print(f'test_loss: {test_loss / len(test_loader):.4f}')
  print(f'test_accuracy: {100 * correct / total:.2f}%')

In [ ]:
train()

학습은 코랩으로 진행했습니다

In [8]:
test()

  0%|          | 0/28 [00:00<?, ?it/s]

test_loss: 1.4833
test_accuracy: 67.90%


-![nn](result/mymodel3_result/acc.JPG)


-![nn](result/mymodel3_result/loss_val.JPG)


-![nn](result/mymodel3_result/loss_train.JPG)